In [1]:
from tqdm.notebook import trange, tqdm
import os

import numpy as np
import pandas as pd
import swifter

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

os.chdir("..")
from src.utils import BETA_STR
from src.config import Ns, betas

In [2]:
def ingest_noise_data(path):
    df_list = []
    files = os.listdir(path)
    for i in range(len(files)):
        noise = float(os.path.splitext(files[i])[0].split("=")[1])
        filename = os.path.join(path, files[i])
        data = pd.read_csv(filename)
        data = data.rename_axis(index="N", columns="beta_ind").melt(ignore_index=False, value_name="lambda").reset_index()
        data["beta_ind"] = data["beta_ind"].astype(np.int32)
        data["noise"] = noise
        data["betas"] = data["beta_ind"].apply(lambda i: betas[i])
        data["N"] += 1 # N range is from 1 to 50 and not 0 to 49
        df_list.append(data)
    return pd.concat(df_list, ignore_index=True)

In [3]:
data = ingest_noise_data("data/ib_to_decay/")

In [4]:
def plot_beta_vs_lambda_n_granularity(ns, noises):
    """
    subplots of different noises. n is used as color
    ns and noises are lists. It is recommended to use no more than 10 values in noises.
    """
    df_to_plot = data.loc[
        (data["noise"].isin(noises)) &
        (data["N"].isin(ns))
    ]
    fig = px.line(df_to_plot, x="betas", y="lambda", color="N", markers=True, 
            title=f"Lambda Regularization vs {BETA_STR}", labels = {"betas":BETA_STR},
            facet_col="noise", facet_col_wrap=2, height=1500, width=1500, facet_col_spacing=0.05)

    fig.update_xaxes(showticklabels=True)
    fig.update_yaxes(showticklabels=True)
    fig.update_xaxes(matches=None)
    fig.update_yaxes(matches=None)
    
    return fig

In [5]:
noises = [0.0,0.01,0.1,0.5,1.,1.5, 3.0, 5.0]
ns = [1, 2, 3, 5 ,10, 20, 30, 40, 49]
fig = plot_beta_vs_lambda_n_granularity(ns, noises)
fig.write_html("plots/lambda_vs_beta_sublpots.html")

In [6]:
def plot_beta_vs_lambda(data):
    """
    n: int or None - multiple noises. List - multiple n's traces on same noise
    noise: int. must be specified if n is List.
    
    """
    title = f"Lambda Regularization vs {BETA_STR} (halflives are averaged)"
    df_to_plot = data.groupby(["betas", "noise"], as_index=False)["lambda"].mean()
    fig = px.line(df_to_plot, x="betas", y="lambda", color="noise", markers=True, 
                  title=title,
                 labels = {"betas":BETA_STR})

    return fig

fig = plot_beta_vs_lambda(data)
fig.write_html("plots/lambda_vs_beta.html")